In [1]:
%reload_ext autoreload
%autoreload 2

In [50]:
from catboost import CatBoostClassifier
import catboost 

import pandas as pd
import json
from pandas import json_normalize
from tqdm import tqdm_notebook as tqdm
import numpy as np
import ast
import json
import re

import sys
import os
sys.path.append(os.path.join(sys.path[0], '../../'))

from core.train_utils import calculate_roi, get_winner_favorite, combine_df

pd.set_option('display.precision',6)
pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None
print(pd.__version__, catboost.__version__, np.__version__)

1.2.4 0.22 1.20.2


In [51]:
with open('./data_models/generated_features_11.05.2021.txt', 'r') as outfile:
    generated_features = json.load(outfile)
fighter1_stats = generated_features['fighter1_stats']
fighter2_stats = generated_features['fighter2_stats']
difference_cols = generated_features['difference_cols']

cat_cols = ['city', 'country', 'is_fight_night', 'weightCategory.id', 
              'f1_country', 'f2_country', 'f1_city', 'f2_city']

#  fighter1_stats + fighter2_stats + 
train_cols = cat_cols + difference_cols + fighter1_stats + fighter2_stats  #+ ['f1_odds', 'f2_odds', 'odds_difference']# +  #['f1_odds', 'f2_odds', 'odds_difference'] +

In [55]:
df = pd.read_csv('./data_models/events_joined_total_11.05.2021.csv', index_col=0)
df = df.rename(columns = lambda x: x.strip())

df['eventDate.date'] = pd.to_datetime(df['eventDate.date'])
df['weightCategory.id'] = df['weightCategory.id'].astype('object')
df['winner'] = df['winnerId'] == df['fighterId_1']
print(df.shape)

df[cat_cols] = df[cat_cols].fillna('unknown')
df = df.fillna(-999)

df = df[(df['f1_count_of_fights'] >= 1) & (df['f2_count_of_fights'] >= 1)]

train1 = df[(df['eventDate.date'].dt.year > 2005) & (df['eventDate.date'].dt.year < 2020) &
           ((df['type']=='straight'))]

val1  = df[(df['eventDate.date'].dt.year == 2020) & (df['eventDate.date'].dt.year < 2021) &
          ((df['type']=='straight'))]
val1 = val1.reset_index()
val1 = val1[~val1['odds_difference'].isna()]
# 'f1_timezone', 'f2_timezone', 'timezone', 'f1_name', 'f2_name', 


train2 = df[(df['eventDate.date'].dt.year > 2005) & (df['eventDate.date'].dt.year < 2020) &
           ((df['type']=='reversed'))]

val2  = df[(df['eventDate.date'].dt.year == 2020) & (df['eventDate.date'].dt.year < 2021) &
          ((df['type']=='reversed'))]
val2 = val2.reset_index()
val2 = val2[~val2['odds_difference'].isna()]

train = train1.append(train2)
val = val1.append(val2)

print(train1.shape, val1.shape)
print(train2.shape, val2.shape)
print(train.shape, val.shape)

test1  = df[(df['eventDate.date'].dt.year == 2021) & 
          ((df['type']=='straight'))]
test1 = test1.reset_index()
test1 = test1[~test1['odds_difference'].isna()]

test2  = df[(df['eventDate.date'].dt.year == 2021) & 
          ((df['type']=='reversed'))]
test2 = test2.reset_index()
test2 = test2[~test2['odds_difference'].isna()]

test1.shape

(11854, 343)
(3565, 343) (338, 344)
(3565, 343) (338, 344)
(7130, 343) (676, 344)


(108, 344)

In [68]:
cat_features = train[train_cols].select_dtypes('object').columns.values.tolist()

def create_cat_feature_dict(df, col):
    thresh = df[col].value_counts().quantile(0.3) 
    temp_df = pd.DataFrame((df[col].value_counts() > 20)).reset_index()

    dict_ = {}
    for num, (raw_value, thresh_status) in enumerate(temp_df.values, start=1):
        if thresh_status:
            dict_[raw_value] = num
    return dict_

city_dict = {col: num for num, col in enumerate(train['city'].unique())}
country_dict = {col: num for num, col in enumerate(train['country'].unique())}
f1_country_dict = {col: num for num, col in enumerate(train['f1_country'].unique())}
f2_country_dict = {col: num for num, col in enumerate(train['f2_country'].unique())}
f1_city_dict = {col: num for num, col in enumerate(train['f1_city'].unique())}
f2_city_dict = {col: num for num, col in enumerate(train['f2_city'].unique())}

for cat_featuer in cat_features:
    dict_ = create_cat_feature_dict(train, cat_featuer)
    train[cat_featuer] = train[cat_featuer].map(city_dict).fillna(0).astype(int)
    val[cat_featuer] = val[cat_featuer].map(city_dict).fillna(0).astype(int)

    
X_train = train[train_cols]#.drop(['f1_odds', 'f2_odds', 'odds_difference'], axis=1)
y_train = train['winner'].astype(int)

X_val = val[train_cols]#.drop(['f1_odds', 'f2_odds', 'odds_difference'], axis=1)
y_val = val['winner'].astype(int)
X_train[cat_features] = X_train[cat_features].astype('category')

# Feature selection

In [69]:
y_train.shape, X_train.shape

((7130,), (7130, 323))

In [70]:
y_val.shape, X_val.shape

((676,), (676, 323))

In [73]:
from shaphypetune import BoostSearch, BoostBoruta, BoostRFE
from lightgbm import LGBMClassifier

import re

X_train = X_train.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
X_val = X_val.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
    
clf_lgbm = LGBMClassifier(n_estimators=1000, random_state=0, n_jobs=-1)#, categorical_feature=[f'name:{i}' for i in cat_features])
clf_lgbm.fit(X_train, y_train.values, eval_set=[(X_val, y_val.values)], early_stopping_rounds=6, verbose=0)



LGBMClassifier(n_estimators=1000, random_state=0)

In [76]:
clf_lgbm.score(X_train, y_train)

0.6678821879382889

In [77]:
clf_lgbm.score(X_val, y_val)

0.5088757396449705

In [ ]:
model = BoostBoruta(clf_lgbm, max_iter=200, perc=100)
model.fit(X_train, y_train.values, eval_set=[(X_val, y_val.values)], early_stopping_rounds=6, verbose=0)


In [23]:
len(y_train), len(X_train)

(7130, 7130)

In [20]:
y_train.values

array([0, 0, 1, ..., 0, 1, 0])